In [1]:
import cudf
from cuml import make_regression
from cuml import train_test_split as train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import pandas as pd

In [2]:
df = pd.read_csv('data/agricultural_data.csv')

df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,1.594896
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,0.500000
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,3.147059
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,3.642045
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,0.229167


In [3]:
df = df[df['State_Name'] == "Andhra Pradesh"]
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
203,Andhra Pradesh,ANANTAPUR,1997,Kharif,Arhar/Tur,21400.0,2600.0,0.121495
204,Andhra Pradesh,ANANTAPUR,1997,Kharif,Bajra,1400.0,500.0,0.357143
205,Andhra Pradesh,ANANTAPUR,1997,Kharif,Castor seed,1000.0,100.0,0.100000
206,Andhra Pradesh,ANANTAPUR,1997,Kharif,Cotton(lint),7300.0,9400.0,1.287671
207,Andhra Pradesh,ANANTAPUR,1997,Kharif,Dry chillies,3700.0,7100.0,1.918919


In [4]:
df = df[df['Crop_Year']>=2004]
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
493,Andhra Pradesh,ANANTAPUR,2004,Kharif,Arhar/Tur,32752.0,7893.0,0.240993
494,Andhra Pradesh,ANANTAPUR,2004,Kharif,Bajra,1021.0,403.0,0.394711
495,Andhra Pradesh,ANANTAPUR,2004,Kharif,Castor seed,1578.0,923.0,0.584918
496,Andhra Pradesh,ANANTAPUR,2004,Kharif,Cotton(lint),8933.0,7041.0,0.788201
497,Andhra Pradesh,ANANTAPUR,2004,Kharif,Dry chillies,3402.0,11288.0,3.318048


In [5]:
df = df.fillna(df.mean())
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
493,Andhra Pradesh,ANANTAPUR,2004,Kharif,Arhar/Tur,32752.0,7893.0,0.240993
494,Andhra Pradesh,ANANTAPUR,2004,Kharif,Bajra,1021.0,403.0,0.394711
495,Andhra Pradesh,ANANTAPUR,2004,Kharif,Castor seed,1578.0,923.0,0.584918
496,Andhra Pradesh,ANANTAPUR,2004,Kharif,Cotton(lint),8933.0,7041.0,0.788201
497,Andhra Pradesh,ANANTAPUR,2004,Kharif,Dry chillies,3402.0,11288.0,3.318048


In [6]:
#df['District_Name'] = df['District_Name'].astype('category')
#df['Crop_Year'] = df['Crop_Year'].astype('category')
#df['Season'] = df['Season'].astype('category')
#df['Crop'] = df['Crop'].astype('category')
#df = df.drop('State_Name', axis=1)

In [6]:
df = df.join(pd.get_dummies(df['District_Name']))
df = df.join(pd.get_dummies(df['Season']))
df = df.join(pd.get_dummies(df['Crop']))
df = df.join(pd.get_dummies(df['Crop_Year']))
df = df.join(pd.get_dummies(df['State_Name']))
df.head

<bound method NDFrame.head of           State_Name  District_Name  Crop_Year       Season          Crop  \
493   Andhra Pradesh      ANANTAPUR       2004  Kharif          Arhar/Tur   
494   Andhra Pradesh      ANANTAPUR       2004  Kharif              Bajra   
495   Andhra Pradesh      ANANTAPUR       2004  Kharif        Castor seed   
496   Andhra Pradesh      ANANTAPUR       2004  Kharif       Cotton(lint)   
497   Andhra Pradesh      ANANTAPUR       2004  Kharif       Dry chillies   
...              ...            ...        ...          ...           ...   
9826  Andhra Pradesh  WEST GODAVARI       2014  Rabi              Tobacco   
9827  Andhra Pradesh  WEST GODAVARI       2014  Rabi               Tomato   
9828  Andhra Pradesh  WEST GODAVARI       2014  Rabi                 Urad   
9829  Andhra Pradesh  WEST GODAVARI       2014  Whole Year       Coconut    
9830  Andhra Pradesh  WEST GODAVARI       2014  Whole Year           Rice   

          Area   Production       Yield  ANAN

In [7]:
df=df.drop('District_Name', axis=1)
df = df.drop('Season',axis=1)
df = df.drop('Crop',axis=1)
df = df.drop('Crop_Year', axis=1)
df = df.drop('Production', axis=1)
df = df.drop('State_Name', axis=1)
df.head

<bound method NDFrame.head of           Area       Yield  ANANTAPUR  CHITTOOR  EAST GODAVARI  GUNTUR  \
493    32752.0    0.240993          1         0              0       0   
494     1021.0    0.394711          1         0              0       0   
495     1578.0    0.584918          1         0              0       0   
496     8933.0    0.788201          1         0              0       0   
497     3402.0    3.318048          1         0              0       0   
...        ...         ...        ...       ...            ...     ...   
9826   28046.0  134.163705          0         0              0       0   
9827     224.0  134.163705          0         0              0       0   
9828    5418.0  134.163705          0         0              0       0   
9829   21729.0  134.163705          0         0              0       0   
9830  409286.0  134.163705          0         0              0       0   

      KADAPA  KRISHNA  KURNOOL  PRAKASAM  ...  2006  2007  2008  2009  2010  \
49

In [8]:
cdf = cudf.DataFrame.from_pandas(df)

In [9]:
cdf.head()

,Area,Yield,ANANTAPUR,CHITTOOR,EAST GODAVARI,GUNTUR,KADAPA,KRISHNA,KURNOOL,PRAKASAM,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,Andhra Pradesh
493,32752.0,0.240993,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
494,1021.0,0.394711,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
495,1578.0,0.584918,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
496,8933.0,0.788201,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
497,3402.0,3.318048,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
X = cdf.drop('Yield', axis = 1)
y = cdf['Yield']

In [11]:
X

,Area,ANANTAPUR,CHITTOOR,EAST GODAVARI,GUNTUR,KADAPA,KRISHNA,KURNOOL,PRAKASAM,SPSR NELLORE,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,Andhra Pradesh
493,32752.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
494,1021.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
495,1578.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
496,8933.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
497,3402.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9826,28046.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9827,224.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9828,5418.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9829,21729.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [12]:
y

493       0.240993
494       0.394711
495       0.584918
496       0.788201
497       3.318048
           ...    
9826    134.163705
9827    134.163705
9828    134.163705
9829    134.163705
9830    134.163705
Name: Yield, Length: 5999, dtype: float64

In [13]:
X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=23)

In [14]:
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## Sklearn

In [15]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 76.5 ms, sys: 45.2 ms, total: 122 ms
Wall time: 31.6 ms


LinearRegression(n_jobs=-1, normalize=True)

In [16]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 23.1 ms, sys: 26.4 ms, total: 49.5 ms
Wall time: 16 ms


In [17]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 7.61 ms, sys: 14.2 ms, total: 21.8 ms
Wall time: 31.9 ms


## cuML

In [20]:
%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              normalize=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 778 ms, sys: 190 ms, total: 969 ms
Wall time: 1.41 s


LinearRegression(algorithm='eig', fit_intercept=True, normalize=True, handle=<cuml.common.handle.Handle object at 0x7fb45c3fa310>, verbose=4, output_type='cudf')

In [21]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 16.8 ms, sys: 2.74 ms, total: 19.5 ms
Wall time: 29.2 ms


In [22]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 460 µs, sys: 8.22 ms, total: 8.68 ms
Wall time: 7.82 ms


In [23]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  0.13756939589409467
R^2 score (cuML): 0.138855741891717
